## Vector Search with IRIS SQL
This tutorial covers how to use InterSystems IRIS as vector storage for the same set of healthcare data that we loaded and vectorized in steps 1A and/or 1B.

 With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the healtchare notes. In addition, we'll be able to apply filters on columns with structured data.

Begin by running the block of code below, which imports the necessary components to get started.

In [2]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

from dotenv import load_dotenv
load_dotenv(override=True)

/opt/homebrew/anaconda3/envs/gs24ws/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

Next, we will set InterSystems IRIS-specific information such as username, password, the hostname and port of the InterSystems IRIS container in this lab, the namespace, and a connection string putting all of those elements together.

In [3]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '55665' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

Using the the connection string we just built, let's establish a connection to InterSystems IRIS.

In [4]:
engine = create_engine(CONNECTION_STRING)

### Load healthcare note data
Next, we will load the JSON file with healthcare notes into a Pandas DataFrame that can be easily imported into InterSystems IRIS as a SQL table.

In [5]:
# Load JSONL file into DataFrame
file_path = './data/healthcare/augmented_notes_100.jsonl'
df_notes = pd.read_json(file_path, lines=True)

In [6]:
df_notes.head()

,note
0,"A a sixteen year-old girl, presented to our Ou..."
1,This is the case of a 56-year-old man that was...
2,A 36-year old female patient visited our hospi...
3,A 49-year-old male presented with a complaint ...
4,A 47-year-old male patient was referred to the...


With the new release of InterSystems IRIS vector search capability, InterSystems IRIS supports vectors as a datatype in tables! In the block below, we will create a table with a few different columns. The last column, 'note_vector', will be used to store vectors that are generated by passing the 'note' of a tweet through an embedding model.

Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, 'description_vector', will be used to store vectors that are generated by passing the 'description' of a review through an embedding model.

In [23]:
with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                DROP TABLE augmented_notes
        """
        result = conn.execute(text(sql))

In [24]:

with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE TABLE augmented_notes (
                note VARCHAR(10000),
                note_vector VECTOR(DOUBLE, 384))
        """
        result = conn.execute(text(sql))

In [8]:
# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [11]:

# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df_notes['note'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df_notes['note_vector'] = embeddings.tolist()


In [12]:
df_notes.head()

,note,note_vector
0,"A a sixteen year-old girl, presented to our Ou...","[-0.00376868830062449, 0.03895513713359833, -0..."
1,This is the case of a 56-year-old man that was...,"[0.05723404884338379, 0.041176751255989075, 0...."
2,A 36-year old female patient visited our hospi...,"[-0.062391310930252075, 0.041186556220054626, ..."
3,A 49-year-old male presented with a complaint ...,"[0.007449138443917036, -0.02825925126671791, -..."
4,A 47-year-old male patient was referred to the...,"[-0.041326187551021576, 0.007342521566897631, ..."


In [25]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df_notes.iterrows():
            sql = text("""
                INSERT INTO augmented_notes 
                (note, note_vector) 
                VALUES (:note, TO_VECTOR(:note_vector))
            """)
            conn.execute(sql, {
                'note': row['note'], 
                'note_vector': str(row['note_vector'])
            })


Let's look for a scotch that costs less than $100, and has an earthy and creamy taste.

In [26]:
note_search = "abdominal pain"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector

In [27]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * FROM augmented_notes
            ORDER BY VECTOR_DOT_PRODUCT(note_vector, TO_VECTOR(:search_vector)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()


In [28]:
print(results)

[('A 38 year old Vietnamese man was admitted with a 10 month history of epigastric pain, fatigue, 10 kg weight loss and recent onset jaundice. He had no ... (1894 characters truncated) ... as performed to debulk the tumour and an ileotransverse anastamosis formed. Due to the extent of the disease and associated abscess the anterior wall', '.0033501861616969108581,-.045345332473516464233,-.028673684224486351013,-.011617254465818405151,-.010714168660342693328,-.091035731136798858642,.0060 ... (8839 characters truncated) ... 63652801,.040151026099920272827,.056639246642589569091,-.022480156272649765014,-.011952088214457035064,.018808880820870399476,-.015940267592668533326'), ('An 88-year-old male presented in the outpatient surgical setting with a chief complaint of a right groin bulge that had been present for 6 weeks. He  ... (1796 characters truncated) ... a relaxing incision was then made in the typical transverse fashion in the right lower quadrant through the rectus sheath, and the 

In [29]:
results_df = pd.DataFrame(results, columns=df_notes.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,note
0,"A 38 year old Vietnamese man was admitted with a 10 month history of epigastric pain, fatigue, 10 kg weight loss and recent onset jaundice. He had no other significant medical history. Clinical examination demonstrated anaemia and a tender mass in the right upper quadrant of the abdomen. A computerised tomography (CT) scan of the abdomen revealed a 7 × 5 cm thick-walled, complex mass adjacent to the second part of the duodenum, which contained fluid and air and abutted the hepatic flexure of the colon. The working diagnosis was a collection secondary to a colonic perforation and he was treated with intravenous antibiotics. He improved with conservative management and was discharged a month later for outpatient colonoscopy. The colonoscopy revealed a lesion in the transverse colon, histology of which showed a mucinous adenocarcinoma.\nHe subsequently returned to the Accident and Emergency Department following an upper gastrointestinal bleed. On his second admission, a repeat CT scan again suggested localised colonic perforation with formation of an abscess adjacent to the duodenum, along with thickening of the ascending colon, predominantly centred around the hepatic flexure. Given the clinical presentation and diagnostic uncertainty a diagnostic laparoscopy was performed which revealed a large perforated tumour at the hepatic flexure with ascites and peritoneal tumour nodules. A laparotomy was performed via a transverse incision and following mobilisation of the hepatic flexure, a colonic tumour was found to have invaded the gallbladder and duodenum with an abscess cavity anterior to the second part of the duodenum. At the base of the abscess cavity a large hole was apparent in the second part of the duodenum with malignant tumour invading the duodenum. Given the size of the defect (5 cm × 2 cm) and the presence of tumour it was not possible to resect and form a primary closure of the duodenum. The presence of metastatic spread precluded a curative resection by pancreatoduodenectomy.\nA right hemicolectomy was performed to debulk the tumour and an ileotransverse anastamosis formed. Due to the extent of the disease and associated abscess the anterior wall"
1,"An 88-year-old male presented in the outpatient surgical setting with a chief complaint of a right groin bulge that had been present for 6 weeks. He had sharp pain initially when he first developed the abnormality but had been asymptomatic ever since. He did not recall any inciting factors. He was concerned that a previously repaired right inguinal hernia had recurred from its original tissue repair in 1977. Details of the original right inguinal hernia repair were unknown to the patient, other than no implantation of mesh occurred. On physical examination, a 3 cm × 3 cm firm, nontender mass was palpable in the right groin just lateral to the pubic tubercle. A computed tomography scan of the abdomen and pelvis was performed to elucidate the cause of the mass in his groin (Figs. , , and ). The imaging was relevant for a right inguinal hernia with the appendix present within the sac. Preoperative laboratory testing revealed a white blood cell count of 4.7 × 109/L. The patient elected to proceed with surgical intervention for hernia repair.\nThe patient presented to the hospital setting for his elective right inguinal hernia repair. A classic oblique incision was made in the right groin using the anterior superior iliac spine and pubic tubercle as landmarks. The external oblique aponeurosis was opened and the hernia isolated and examined. The hernia was noted to be comprised of an extremely hard and dense amount of omentum that had a chronic, scarred appearance. The base of the appendix could be seen exiting the internal inguinal ring, but the densely adhered omentum prevented reduction of the appendix back into the peritoneal cavity. Initially, there was no indication to perform an appendectomy at the time of the procedure if the appendix could be successfully reduc